In [1]:
import os
import pandas as pd
import ftplib
import SimpleITK as sitk

In [2]:
def getFile(ftp, base_path, filename):
    try:
        ftp.retrbinary("RETR " + filename ,open(os.path.join(base_path,filename), 'wb').write)
    except:
        print("Error")

In [3]:
output_directory = r'C:\Users\LydiaN\OneDrive - Personal\OneDrive\Work\DigitalAssets\BICCNMorphologyInventory\u19zeng'
inventory_path = os.path.join(output_directory,'sample_inventory_with_image_info.csv')
inventory = pd.read_csv(inventory_path)
inventory.set_index('Sample ID', inplace=True)

inventory.head()

,HUST Sample ID,Sample Type (CV),Species (CV),Species NCBI Taxonomy ID,Parent Specimen ID,Parent Specimen Type (CV),Subject ID,Age,Sex (CV),Genotype,...,Directories,TXT Files,JSON Files,x_resolution,y_resolution,z_resolution,x_size,y_size,z_size,rotation
Sample ID,,,,,,,,,,,,,,,,,,,,,
mouseID_210254-15257,15257,whole brain,mouse,NCBI:txid10090,NaN,NaN,210254,22w,female,Pvalb-T2A-CreERT2/wt;Ai139(TIT2L-GFP-ICL-TPT)/wt,...,.;..;green;red,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mouseID_236174-16124,16124,whole brain,mouse,NCBI:txid10090,NaN,NaN,236174,21w,male,Gnb4-IRES2-CreERT2/wt;Ai139(TIT2L-GFP-ICL-TPT)...,...,.;..;green;red,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mouseID_297974-17109,17109,whole brain,mouse,NCBI:txid10090,NaN,NaN,297974,14w,male,Gnb4-IRES2-CreERT2/wt;Ai140(TIT2L-GFP-ICL-tTA2...,...,.;..;green;red,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mouseID_314107-17300,17300,whole brain,mouse,NCBI:txid10090,NaN,NaN,314107,48w,male,Plxnd1-CreER/wt;Ai82(TITL-GFP)/Ai140(TIT2L-GFP...,...,.;..;green;red,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mouseID_321237-17302,17302,whole brain,mouse,NCBI:txid10090,NaN,NaN,321237,29w,male,Tnnt1-IRES2-CreERT2/wt;Ai82(TITL-GFP)/Ai140(TI...,...,.;..;green;red,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
cols = ['sample_id', 'hust_imaging_id', 'labtrack_mouse_id', 'channel', 'image_type', 'resolution', 'number_of_images', 'first_image_path', 'last_image_path']
file_inventory = pd.DataFrame( columns = cols )

In [5]:
def get_dir_info(ftp) :
    
    data = []
    ftp.dir(data.append)
    data = [x.split() for x in data if x != '']
    data = pd.DataFrame(data)
    
    return data

def check_for_directory(data,dirname) :
    has_dir = False
    for dindex, drow in data.iterrows() :
        if drow[8] == dirname :
            has_dir = True
    return has_dir

In [6]:
count = 0
for iindex, irow in inventory.iterrows() :
        
    ftp = ftplib.FTP(host="download.brainimagelibrary.org")
    ftp.login()

    home_dir = ftp.pwd()
    remote_base = "/bil/data/biccn/zeng/luo/fMOST"
    
    print(iindex)
    
    # specimen output directory
    spath = os.path.join(output_directory,iindex)
    
    # get listing of remote directory
    rpath = remote_base + "/" + iindex
    
    for c in ['green', 'red'] :
        
        for t in ['raw', 'mip'] :
            
            for r in ['fullres', 'lowres'] :
                
                ftp.cwd( rpath )
                dlist = [c,t,r]
                print( "- " + str(dlist))
    
                for d in dlist :
                    data = get_dir_info(ftp)
                    has_dir = check_for_directory( data, d )
                    if not has_dir :
                        break
                    ftp.cwd(d)
        
                if not has_dir :
                    print("-- Directory not found - skipping")
                    continue        
    
                rec = {}
                rec['sample_id'] = iindex
                rec['hust_imaging_id'] = irow['HUST Sample ID']
                rec['labtrack_mouse_id'] = irow['Subject ID']
                rec['channel'] = c
                rec['image_type'] = t
                rec['resolution'] = r
 
                if 0 :
                    print("-- Start: dir listing")
                    data = get_dir_info(ftp)
                    print("-- End: dir listing")

                    matched = ['tif' in x for x in data[8]]
                    filtered = data[matched]
                    ordered = filtered.sort_values(by=8)
                    if len(ordered) > 0 :
                        first_file = ordered.loc[ordered.index[0], 8]
                        last_file = ordered.loc[ordered.index[-1], 8]
                        num_files = len(ordered)
                        print(num_files, first_file, last_file)
                    
                        rec['number_of_images'] = num_files
                        rec['first_image_path'] = first_file
                        rec['last_image_path'] = last_file
                    
                file_inventory = file_inventory.append(rec, ignore_index=True)

    ftp.quit()


mouseID_210254-15257
- ['green', 'raw', 'fullres']
- ['green', 'raw', 'lowres']
-- Directory not found - skipping
- ['green', 'mip', 'fullres']
- ['green', 'mip', 'lowres']
-- Directory not found - skipping
- ['red', 'raw', 'fullres']
- ['red', 'raw', 'lowres']
-- Directory not found - skipping
- ['red', 'mip', 'fullres']
- ['red', 'mip', 'lowres']
-- Directory not found - skipping
mouseID_236174-16124
- ['green', 'raw', 'fullres']
- ['green', 'raw', 'lowres']
-- Directory not found - skipping
- ['green', 'mip', 'fullres']
-- Directory not found - skipping
- ['green', 'mip', 'lowres']
-- Directory not found - skipping
- ['red', 'raw', 'fullres']
- ['red', 'raw', 'lowres']
-- Directory not found - skipping
- ['red', 'mip', 'fullres']
-- Directory not found - skipping
- ['red', 'mip', 'lowres']
-- Directory not found - skipping
mouseID_297974-17109
- ['green', 'raw', 'fullres']
- ['green', 'raw', 'lowres']
-- Directory not found - skipping
- ['green', 'mip', 'fullres']
-- Directory not f

- ['green', 'mip', 'lowres']
- ['red', 'raw', 'fullres']
- ['red', 'raw', 'lowres']
- ['red', 'mip', 'fullres']
-- Directory not found - skipping
- ['red', 'mip', 'lowres']
-- Directory not found - skipping
mouseID_374707-18452
- ['green', 'raw', 'fullres']
- ['green', 'raw', 'lowres']
- ['green', 'mip', 'fullres']
- ['green', 'mip', 'lowres']
- ['red', 'raw', 'fullres']
- ['red', 'raw', 'lowres']
- ['red', 'mip', 'fullres']
-- Directory not found - skipping
- ['red', 'mip', 'lowres']
-- Directory not found - skipping
mouseID_375394-18468
- ['green', 'raw', 'fullres']
- ['green', 'raw', 'lowres']
- ['green', 'mip', 'fullres']
- ['green', 'mip', 'lowres']
- ['red', 'raw', 'fullres']
- ['red', 'raw', 'lowres']
- ['red', 'mip', 'fullres']
- ['red', 'mip', 'lowres']
mouseID_377387-18466
- ['green', 'raw', 'fullres']
- ['green', 'raw', 'lowres']
- ['green', 'mip', 'fullres']
- ['green', 'mip', 'lowres']
- ['red', 'raw', 'fullres']
- ['red', 'raw', 'lowres']
- ['red', 'mip', 'fullres']
-- Di

- ['green', 'mip', 'fullres']
- ['green', 'mip', 'lowres']
- ['red', 'raw', 'fullres']
- ['red', 'raw', 'lowres']
- ['red', 'mip', 'fullres']
-- Directory not found - skipping
- ['red', 'mip', 'lowres']
-- Directory not found - skipping
mouseID_486478-196478
- ['green', 'raw', 'fullres']
- ['green', 'raw', 'lowres']
- ['green', 'mip', 'fullres']
- ['green', 'mip', 'lowres']
- ['red', 'raw', 'fullres']
-- Directory not found - skipping
- ['red', 'raw', 'lowres']
-- Directory not found - skipping
- ['red', 'mip', 'fullres']
-- Directory not found - skipping
- ['red', 'mip', 'lowres']
mouseID_373373-196484
- ['green', 'raw', 'fullres']
- ['green', 'raw', 'lowres']
- ['green', 'mip', 'fullres']
- ['green', 'mip', 'lowres']
- ['red', 'raw', 'fullres']
-- Directory not found - skipping
- ['red', 'raw', 'lowres']
-- Directory not found - skipping
- ['red', 'mip', 'fullres']
-- Directory not found - skipping
- ['red', 'mip', 'lowres']
mouseID_384979-196485
- ['green', 'raw', 'fullres']
- ['gre

-- Directory not found - skipping
- ['red', 'mip', 'fullres']
-- Directory not found - skipping
- ['red', 'mip', 'lowres']
mouseID_467364-194067
- ['green', 'raw', 'fullres']
- ['green', 'raw', 'lowres']
- ['green', 'mip', 'fullres']
- ['green', 'mip', 'lowres']
- ['red', 'raw', 'fullres']
-- Directory not found - skipping
- ['red', 'raw', 'lowres']
-- Directory not found - skipping
- ['red', 'mip', 'fullres']
-- Directory not found - skipping
- ['red', 'mip', 'lowres']
mouseID_467369-194068
- ['green', 'raw', 'fullres']
- ['green', 'raw', 'lowres']
- ['green', 'mip', 'fullres']
- ['green', 'mip', 'lowres']
- ['red', 'raw', 'fullres']
-- Directory not found - skipping
- ['red', 'raw', 'lowres']
-- Directory not found - skipping
- ['red', 'mip', 'fullres']
-- Directory not found - skipping
- ['red', 'mip', 'lowres']
mouseID_454426-192342
- ['green', 'raw', 'fullres']
- ['green', 'raw', 'lowres']
- ['green', 'mip', 'fullres']
- ['green', 'mip', 'lowres']
- ['red', 'raw', 'fullres']
-- Di

mouseID_486477-196477
- ['green', 'raw', 'fullres']
- ['green', 'raw', 'lowres']
- ['green', 'mip', 'fullres']
- ['green', 'mip', 'lowres']
- ['red', 'raw', 'fullres']
-- Directory not found - skipping
- ['red', 'raw', 'lowres']
-- Directory not found - skipping
- ['red', 'mip', 'fullres']
-- Directory not found - skipping
- ['red', 'mip', 'lowres']
mouseID_487259-196479
- ['green', 'raw', 'fullres']
- ['green', 'raw', 'lowres']
- ['green', 'mip', 'fullres']
- ['green', 'mip', 'lowres']
- ['red', 'raw', 'fullres']
-- Directory not found - skipping
- ['red', 'raw', 'lowres']
-- Directory not found - skipping
- ['red', 'mip', 'fullres']
-- Directory not found - skipping
- ['red', 'mip', 'lowres']
mouseID_487260-196480
- ['green', 'raw', 'fullres']
- ['green', 'raw', 'lowres']
- ['green', 'mip', 'fullres']
- ['green', 'mip', 'lowres']
- ['red', 'raw', 'fullres']
-- Directory not found - skipping
- ['red', 'raw', 'lowres']
-- Directory not found - skipping
- ['red', 'mip', 'fullres']
-- Di

mouseID_491995-196473
- ['green', 'raw', 'fullres']
- ['green', 'raw', 'lowres']
- ['green', 'mip', 'fullres']
- ['green', 'mip', 'lowres']
- ['red', 'raw', 'fullres']
-- Directory not found - skipping
- ['red', 'raw', 'lowres']
-- Directory not found - skipping
- ['red', 'mip', 'fullres']
-- Directory not found - skipping
- ['red', 'mip', 'lowres']
mouseID_493297-196474
- ['green', 'raw', 'fullres']
- ['green', 'raw', 'lowres']
- ['green', 'mip', 'fullres']
- ['green', 'mip', 'lowres']
- ['red', 'raw', 'fullres']
-- Directory not found - skipping
- ['red', 'raw', 'lowres']
-- Directory not found - skipping
- ['red', 'mip', 'fullres']
-- Directory not found - skipping
- ['red', 'mip', 'lowres']
mouseID_511266-201604
- ['green', 'raw', 'fullres']
- ['green', 'raw', 'lowres']
- ['green', 'mip', 'fullres']
- ['green', 'mip', 'lowres']
- ['red', 'raw', 'fullres']
-- Directory not found - skipping
- ['red', 'raw', 'lowres']
-- Directory not found - skipping
- ['red', 'mip', 'fullres']
-- Di

In [7]:
file_inventory_path = os.path.join( output_directory, 'file_inventory_part_2.csv')
file_inventory.to_csv( file_inventory_path, index=False )

In [10]:
file_inventory

,sample_id,hust_imaging_id,labtrack_mouse_id,channel,image_type,resolution,number_of_images,first_image_path,last_image_path
0,mouseID_210254-15257,15257,210254,green,raw,fullres,NaN,NaN,NaN
1,mouseID_210254-15257,15257,210254,green,mip,fullres,NaN,NaN,NaN
2,mouseID_210254-15257,15257,210254,red,raw,fullres,NaN,NaN,NaN
3,mouseID_210254-15257,15257,210254,red,mip,fullres,NaN,NaN,NaN
4,mouseID_236174-16124,16124,236174,green,raw,fullres,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
716,mouseID_442529-191811,191811,442529,green,raw,fullres,NaN,NaN,NaN
717,mouseID_442529-191811,191811,442529,green,raw,lowres,NaN,NaN,NaN
718,mouseID_442529-191811,191811,442529,green,mip,fullres,NaN,NaN,NaN
719,mouseID_442529-191811,191811,442529,green,mip,lowres,NaN,NaN,NaN
